In [ ]:
from fastai.vision import *
from fastai.callbacks import *
from fastai.metrics import error_rate
import torch.nn as nn
import pandas as pd
import shutil
from geffnet import tf_mobilenetv3_large_minimal_100
from geffnet import tf_mobilenetv3_small_minimal_100 
from geffnet import efficientnet_b0
from geffnet import efficientnet_b1
from geffnet import efficientnet_b2
from geffnet import efficientnet_b3
from geffnet import tf_mobilenetv3_small_100
from geffnet import tf_mobilenetv3_small_075
from geffnet import mobilenetv3_large_100
from geffnet import mobilenetv2_120d
from geffnet import mobilenetv2_110d
import os
torch.cuda.set_device(0)

In [ ]:
### Training ###

model = mobilenetv2_120d(pretrained=True, num_classes = 20)
for param in model.parameters():
    param.requires_grad = False
model.classifier=nn.Sequential(
        nn.Linear(1280, out_features=20, bias=False),
        nn.LogSoftmax())
UCSD_path = './'
tfms = get_transforms(flip_vert=True, max_lighting=0.1, max_zoom=1.1, max_warp=0.)

data_UCSD = ImageDataBunch.from_folder(path=UCSD_path, train='train', valid='val', ds_tfms=tfms, 
            test=None, valid_pct=None, no_check=True, size=(224,224), num_workers=6, bs=280)
#opt = optim.Adam
data_UCSD = data_UCSD.normalize(imagenet_stats)
learn = Learner(data_UCSD, model, metrics=[error_rate, accuracy, top_k_accuracy], callback_fns=[CSVLogger])
epochs=25

learn.fit_one_cycle(epochs, max_lr=1e-3, callbacks=[SaveModelCallback(learn, 
                            monitor='error_rate', mode='min', name="model")])

#if you want to use fast ai to save model and do mobile, that may occur some problem.
torch.save(model.state_dict(), './models/model.pth')

In [ ]:
### Evaluate ###

img = open_image('./val/frog/1379123151-322948949.jpg')
y=learn.predict(img)
a = torch.argmax(y[2]).numpy()
print(learn.data.classes[a])
learn.validate(data_UCSD.valid_dl,metrics=[error_rate, accuracy, top_k_accuracy])